# info
This script generates a file with filtered DEGs derived from patient-specific DEG calling.

Filters are:
1. Filter on P < 0.05 and Q < 0.1 for DESeq2; Q < 0.05 in sleuth.
2. Filter out genes that in the comparison do not cross the 2 TPM barrier.
3. Discrete abs log2 FC > 1

Coherent DEGs are the ones that pass all three filters in n-1 patients.

In [1]:
import pandas, os, numpy

# 0. user-defined variables

In [2]:
expression_threshold = 2
discrete_fc_threshold = 1
noise_threshold = 1/2

In [3]:
hypotheses = ['hypothesis_A', 'hypothesis_B', 'hypothesis_C', 'hypothesis_D', 'hypothesis_E']
trends = ['up', 'down']

In [4]:
metadata_file = '/home/adrian/projects/hegoi/metadata/hegoi metadata - hypotheses formatted for filter.tsv'
tpm_file = '/home/adrian/projects/hegoi/results/tpm/DESeq2_TPM_values.tsv'
deseq2_folder = '/home/adrian/projects/hegoi/results/subsamples/DESeq2/'

output_dir = '/home/adrian/projects/hegoi/results/subsamples/DEG_filtered/'
filtered_DEGs_file = '/home/adrian/projects/hegoi/results/subsamples/DEG_filtered/filtered_DEGs.tsv'

# 1. read input files

## 1.1 read expression data

In [5]:
expression = pandas.read_csv(tpm_file, sep='\t', index_col=0)
sample_names = expression.columns.to_list()

print(sample_names)
expression.head()

['Lam153', 'Lami154', 'Lami46', 'Lami94', 'LamiP109', 'LamiP153', 'LamiP154', 'LamiP176', 'LamiPi46', 'LamiPi94', 'Osci109', 'Osci153', 'Osci154', 'Osci46', 'OsciP109', 'OsciP153', 'OsciP154', 'OsciP175', 'OsciP178', 'OsciPi46', 'Stat109', 'Stat153', 'Stat154', 'Stat176', 'Stat46', 'Stat94']


,Lam153,Lami154,Lami46,Lami94,LamiP109,LamiP153,LamiP154,LamiP176,LamiPi46,LamiPi94,...,OsciP154,OsciP175,OsciP178,OsciPi46,Stat109,Stat153,Stat154,Stat176,Stat46,Stat94
ENSG00000000003,24.240953,20.242011,22.919919,15.438227,18.098457,29.000777,14.784503,11.903633,26.217884,18.455471,...,38.756393,15.976121,21.649331,28.426877,29.688663,29.207002,28.989493,28.832380,34.600089,27.971771
ENSG00000000005,0.062960,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013958,0.000000,0.139231
ENSG00000000419,59.982918,62.984443,120.871295,138.059565,115.707396,84.892000,172.643218,147.901176,145.183145,135.750959,...,98.473591,161.024012,86.023654,115.321222,64.464680,81.686544,84.388415,88.740844,81.992995,97.987628
ENSG00000000457,4.680221,3.684855,1.989328,3.244174,4.417648,2.189278,6.217483,3.998143,4.786793,2.949599,...,3.012943,1.024624,2.255165,2.967113,1.825466,2.371161,3.513856,2.617785,2.546863,2.668982
ENSG00000000460,1.982690,1.344508,1.422365,1.667403,0.570064,2.856698,0.416009,0.000000,1.310974,1.812507,...,5.510741,3.805323,1.489214,0.957885,4.467578,5.425887,7.680067,4.002537,3.650609,4.056977


## 1.2. read metadata

In [6]:
metadata = pandas.read_csv(metadata_file, sep='\t', index_col=0)
metadata.head()

,patient,sampleA,sampleB
hypothesis,,,
hypothesis_A,46,Stat46,Lami46
hypothesis_A,153,Stat153,Lam153
hypothesis_A,154,Stat154,Lami154
hypothesis_A,94,Stat94,Lami94
hypothesis_B,46,Lami46,Osci46


# 2. iterate over hypotheses and patients

In [7]:
f = open(filtered_DEGs_file, 'w')
f.write('hypo\ttrend\tensembl\n')

19

In [8]:
annotation = {}

for hypothesis in hypotheses:
    print('working with hypothesis {}...'.format(hypothesis))
    
    ### 1. define patients
    elements = os.listdir(deseq2_folder)
    working_elements = [element for element in elements if hypothesis in element]
    
    ### 2. define consistent DEGs
    for trend in trends:
        
        patient_result_files = [working_element for working_element in working_elements if trend in working_element]
        number_of_patients = len(patient_result_files)
        print(patient_result_files, number_of_patients)
        
        # gather all DEGs per subsample
        subsample_DEGs = []
        for patient_result_file in patient_result_files:
            
            patient = int(patient_result_file.split('_')[2])
            print('\t working with patient {}'.format(patient))
            
            info = pandas.read_csv(deseq2_folder + patient_result_file, sep='\t', index_col=0)
            DEGs = info.index.to_list()
            print('\t\t detected {} DEGs'.format(len(DEGs)))
            for element in DEGs:
                subsample_DEGs.append(element)
                
            # building an annotation in a dictionary
            for element in DEGs:
                if element not in annotation:
                    biotype = info['gene_biotype'].loc[element]
                    
                    if type(info['description'].loc[element]) is not str:
                        description = 'NA'
                    else:
                        description = info['description'].loc[element].split(' [Source')[0]
                        
                    symbol = info['hgnc_symbol'].loc[element]
                    if type(symbol) is not str:
                        symbol = ''
                    
                    annotation[element] = [biotype, symbol, description]
                
        # find the intersect
        consistent_DEGs = []
        for element in subsample_DEGs:
            if subsample_DEGs.count(element) >= number_of_patients - 1:
                consistent_DEGs.append(element)
        consistent_DEGs = list(set(consistent_DEGs))
        print('\t found {} consistent DEGs'.format(len(consistent_DEGs)))    
    
        ### 3. filter on low expression and FC 
        substantial_genes = []; noisy_genes = []
        for gene in consistent_DEGs:
            
            success = 0
            nums = []; dens = []
            
            for patient_result_file in patient_result_files:
                patient = int(patient_result_file.split('_')[2])
                
                ref_label = None; sam_label = None
                ref_label = metadata[metadata['patient'] == patient].loc[hypothesis]['sampleA']
                sam_label = metadata[metadata['patient'] == patient].loc[hypothesis]['sampleB']
                
                tpm_ref = expression[ref_label].loc[gene]
                tpm_sam = expression[sam_label].loc[gene]
                
                top = numpy.max([tpm_ref, tpm_sam])
                
                ###            [round(x, epsilon)/epsilon ] + 1
                ###  FC = abs  -------------------------------- > 1
                ###            [round(y, epsilon)/epsilon ] + 1
                num = numpy.around(tpm_sam) + 1
                den = numpy.around(tpm_ref) + 1
                fc = num/den
                abs_log2FC = numpy.abs(numpy.log2(fc))
                nums.append(num)
                dens.append(den)
                
                # condition
                if (top >= expression_threshold) & (abs_log2FC > discrete_fc_threshold):
                    success = success + 1
            
            # selection
            if success >= number_of_patients - 1:
                substantial_genes.append(gene)
                
            # noise
            sem_ref = numpy.std(dens) / numpy.sqrt(len(dens))
            rsem_ref = sem_ref / numpy.mean(dens)
            
            sem_sam = numpy.std(nums) / numpy.sqrt(len(nums))
            rsem_sam = sem_sam / numpy.mean(nums)
            
            noise = numpy.max([rsem_ref, rsem_sam])
            
            if noise > noise_threshold:
                noisy_genes.append(gene)
                
        # print line
        print('\t found {} substantial genes'.format(len(substantial_genes)))
        
        ### 4. check on noisy genes
        selected_genes = [element for element in substantial_genes if element not in noisy_genes]
        print('\t found {} selected genes'.format(len(selected_genes)))
        
        ###
        print('\t writing to table')
        for gene in selected_genes:
            content = [hypothesis, trend, gene]
            
            string = "\t".join(content)
            f.write(string + '\n')
        
        print()
    print()

working with hypothesis hypothesis_A...
['hypothesis_A_154_up.tsv', 'hypothesis_A_46_up.tsv', 'hypothesis_A_94_up.tsv', 'hypothesis_A_153_up.tsv'] 4
	 working with patient 154
		 detected 894 DEGs
	 working with patient 46
		 detected 573 DEGs
	 working with patient 94
		 detected 905 DEGs
	 working with patient 153
		 detected 853 DEGs
	 found 188 consistent DEGs
	 found 149 substantial genes
	 found 144 selected genes
	 writing to table

['hypothesis_A_154_down.tsv', 'hypothesis_A_46_down.tsv', 'hypothesis_A_94_down.tsv', 'hypothesis_A_153_down.tsv'] 4
	 working with patient 154
		 detected 1034 DEGs
	 working with patient 46
		 detected 894 DEGs
	 working with patient 94
		 detected 1436 DEGs
	 working with patient 153
		 detected 917 DEGs
	 found 361 consistent DEGs
	 found 282 substantial genes
	 found 276 selected genes
	 writing to table


working with hypothesis hypothesis_B...
['hypothesis_B_154_up.tsv', 'hypothesis_B_46_up.tsv', 'hypothesis_B_153_up.tsv'] 3
	 working with pat

In [9]:
f.close()

# 3. prepare a final tables for publication

These are the columns that we will create:

- ENSEMBL
- Gene name
- Biotype
- Description
- log2FC
- Adjusted P-value
- Reference expression (TPM)
- Sample expression (TPM)
- Discrete abs(log2FC)

In [10]:
selection = pandas.read_csv(filtered_DEGs_file, sep='\t', index_col=0)

In [11]:
print(selection.shape)
selection.head()

(999, 2)


,trend,ensembl
hypo,,
hypothesis_A,up,ENSG00000157514
hypothesis_A,up,ENSG00000159261
hypothesis_A,up,ENSG00000178038
hypothesis_A,up,ENSG00000141458
hypothesis_A,up,ENSG00000135407


In [12]:
for hypothesis in hypotheses:
    for trend in trends:
        print(hypothesis, trend)
        
        ### 1. get the genes
        df = selection[selection['trend'] == trend].loc[hypothesis]
        print(df.head())
        print(df.shape)
        if type(df['ensembl']) == str:
            print('WARNING: force converting to list')
            working_genes = [df['ensembl']]
            print('END RESULT:', working_genes)
        else:
            working_genes = df['ensembl'].to_list()
            
        print(len(working_genes), len(list(set(working_genes))))
        
        ### 2. get the statistical information into a dictionary
        statistical_info = {}
        for working_gene in working_genes:
            statistical_info[working_gene] = ([], [])
        
        files = os.listdir(deseq2_folder)
        working_files = [element for element in files if (hypothesis in element) and (trend in element)]
        print(working_files)
        for working_file in working_files:
            f = open(deseq2_folder + working_file, 'r')
            next(f)
            for line in f:
                v = line.split('\t')
                ensembl = v[0]
                log2FC = float(v[2])
                adjusted = float(v[6])
                if ensembl in working_genes:
                    statistical_info[ensembl][0].append(log2FC)
                    statistical_info[ensembl][1].append(adjusted)
            f.close()
            
        ### 3. get quantitative information
        elements = os.listdir(deseq2_folder)
        working_elements = [element for element in elements if hypothesis in element]
        patient_result_files = [working_element for working_element in working_elements if trend in working_element]
        
        quantitative_information = {}
        
        for working_gene in working_genes:
            quantitative_information[working_gene] = ([], [], [])
               
            for patient_result_file in patient_result_files:
                patient = int(patient_result_file.split('_')[2])

                ref_label = None; sam_label = None
                ref_label = metadata[metadata['patient'] == patient].loc[hypothesis]['sampleA']
                sam_label = metadata[metadata['patient'] == patient].loc[hypothesis]['sampleB']

                tpm_ref = expression[ref_label].loc[working_gene]
                tpm_sam = expression[sam_label].loc[working_gene]

                num = numpy.around(tpm_sam) + 1
                den = numpy.around(tpm_ref) + 1
                fc = num/den
                abs_log2FC = numpy.abs(numpy.log2(fc))

                quantitative_information[working_gene][0].append(tpm_ref)
                quantitative_information[working_gene][1].append(tpm_sam)
                quantitative_information[working_gene][2].append(abs_log2FC)
                if working_gene == 'ENSG00000127528':
                    print(hypothesis, trend, abs_log2FC, tpm_ref, tpm_sam, working_gene)
        
        ### 4. write the final output files
        outputfile = output_dir + 'annotated_' + hypothesis + '_' + trend + '.tsv'
        
        f = open(outputfile, 'w')
        f.write('ENSEMBL\tBiotype\tSymbol\tDescription\tlog2FC\tadjusted P\tReference expression (TPM)\tSample expression (TPM)\tDiscrete abs(log2FC)\n')
        
        for element in working_genes:
            
            # start the gene
            f.write('{}\t'.format(element))
            
            # write annotation
            ann = annotation[element]
            string = '\t'.join(ann)
            f.write(string)
            
            # write statistical information
            string = '\t{}\t{}'.format(numpy.median(statistical_info[element][0]), numpy.median(statistical_info[element][1]))
            f.write(string)
            
            # write quantitative information
            string = '\t{}\t{}\t{}'.format(numpy.median(quantitative_information[element][0]), numpy.median(quantitative_information[element][1]), numpy.median(quantitative_information[element][2]))
            f.write(string)
            
            # close the line
            f.write('\n')
        
        f.close()

hypothesis_A up
             trend          ensembl
hypo                               
hypothesis_A    up  ENSG00000157514
hypothesis_A    up  ENSG00000159261
hypothesis_A    up  ENSG00000178038
hypothesis_A    up  ENSG00000141458
hypothesis_A    up  ENSG00000135407
(144, 2)
144 144
['hypothesis_A_154_up.tsv', 'hypothesis_A_46_up.tsv', 'hypothesis_A_94_up.tsv', 'hypothesis_A_153_up.tsv']
hypothesis_A up 3.6948801927991917 38.7095 517.2396 ENSG00000127528
hypothesis_A up 0.5276293256552045 76.27392 109.8661 ENSG00000127528
hypothesis_A up 1.7030182622428685 42.14758 139.3405 ENSG00000127528
hypothesis_A up 3.810137362728931 36.35114 518.4325 ENSG00000127528
hypothesis_A down
             trend          ensembl
hypo                               
hypothesis_A  down  ENSG00000101445
hypothesis_A  down  ENSG00000161800
hypothesis_A  down  ENSG00000106484
hypothesis_A  down  ENSG00000129422
hypothesis_A  down  ENSG00000166960
(276, 2)
276 276
['hypothesis_A_154_down.tsv', 'hypothesis_A_46_

In [17]:
print(expression[['Stat46', 'Stat153', 'Stat154', 'Stat94']].loc['ENSG00000127528'])
print(numpy.median(expression[['Stat46', 'Stat153', 'Stat154', 'Stat94']].loc['ENSG00000127528']))

Stat46     76.27392
Stat153    36.35114
Stat154    38.70950
Stat94     42.14758
Name: ENSG00000127528, dtype: float64
40.42854


In [21]:
print(expression[['Lami46', 'Lam153', 'Lami154', 'Lami94']].loc['ENSG00000127528'])
numpy.median(expression[['Lami46', 'Lam153', 'Lami154', 'Lami94']].loc['ENSG00000127528'])

Lami46     109.8661
Lam153     518.4325
Lami154    517.2396
Lami94     139.3405
Name: ENSG00000127528, dtype: float64


328.29005

In [15]:
numpy.median([2.9385994553358565, 0.0, 3.2479275134435857])

2.9385994553358565

In [16]:
expression.loc['ENSG00000127528']

Lam153      518.43250
Lami154     517.23960
Lami46      109.86610
Lami94      139.34050
LamiP109     36.26593
LamiP153    295.53080
LamiP154     61.98699
LamiP176    124.09520
LamiPi46     78.24828
LamiPi94    155.90690
Osci109      59.79601
Osci153      34.18485
Osci154      34.51338
Osci46       66.71243
OsciP109     26.75359
OsciP153     16.91096
OsciP154     39.70780
OsciP175     23.46088
OsciP178     29.78379
OsciPi46     73.62446
Stat109      24.01949
Stat153      36.35114
Stat154      38.70950
Stat176      26.24223
Stat46       76.27392
Stat94       42.14758
Name: ENSG00000127528, dtype: float64